# Текстовое описание подхода к решению задачи 

Работаю с видео с определенным разрешением (720 x 1280) - это позволяет найти конкретное положение в пикселях для конкретных всплывающих окон.* Поиск координат всплывающих окон происходил вручную.  
  
Всего рассматривалось 4 типа всплывающих окон:  
1) `primary_fighter_info` - краткая информация о бойце (имя, рекорд и страна)  
2) `extensive_fighter_info` - общая информация о бойцах (рекорд, возраст, рост, вес, страна, имя)  
3) `fight_info` - информация о бое. Может быть двух видов: только время боя и раунд или добавлены имена бойцов и их цвет  
4) `winner_or_adv` - реклама или имя победителя  

Для всех всплывающих окон были найдены координаты рамок, в которых содержится извлекаемая информация. `INFO_COORDINATES`
содержит все эти координаты для каждого из окон.  

Чтобы определить, есть ли на изображении всплывающее окно одного из типов, приведенных выше, необходимо иметь некие паттерны, 
по которым можно отличить изображение с всплывающим окном. Можно было решать эту задачу через обучение нейронной сети, но показалось,
что можно найти простые паттерны с использованием знаний об интенсвности пикселей. Предварительно изображения были 
переведены в черно-белые. Так, на изображении были найдены самые яркие участки - это оказались белые вертикальные полосы в случае 1, 3, 4 рамок и горизонтальные полосы в случае 2 типа всплывающего окна.  
Рядом с белыми полосами всегда находится темный участок. По усредненной разнице интенсивностей из "белой" и "темной" полос 
и устанавливается, есть ли на изображении определенное всплывающее окно. Пусть это значение - `mean_intensity`.
Предварительно были найдены `min_intensity` и `max_intensity`, по которым можно будет сказать, что если
`min_intensity < mean_intensity <  max_intensity`, то на изображении есть искомый паттерн всплывающего окна. Поиск значений 
min_intensity и max_intensity происходил так: сначала был создан массив изображений из выбранного видео, он просматривался с помощью napari, и был создан массив, в котором обозначалось, есть ли на изображении всплывающее окно и какого оно типа. 
И для каждой группы изображений с определенным типом всплывающего окна были найдены значения `mean_intensity`. По ним и были установлены
`min_intensity` и `max_intensity`. Координаты паттернов содержатся в `PATTERNS_COORDINATES`, а значения `min_intensity` и `max_intensity` в `PATTERNS_MIN_MAX`.  
  
В видео берётся каждое 25 изображение (fps = 25), то есть дискретизация - 1 изображение на 1 секунду видео.  
Такая дискретизация обусловлена тем, что новая информация может появляться только каждую новую секунду - это информация о секундах боя
во всплывающих окнах `fight_info`.  
Изображение переводится в черно-белое для удобства обработки, вся необходимая информация при этом сохраняется.  
Алгоритм поиска информации о всплывающем окне на изображении:  
1. Поиск - есть ли какое-нибудь всплывающее окно на данном изображении.  
2. Если есть всплывающее окно, то поиск информации в данном окне. Для каждого окна вызывается своя функция поиска нужной информации.  
3. Запись этой информации в словарь, где ключ - это секунда видео.  
  
Поиск текста в рамке происходил с помощью библиотеки для поиска символов - `easyocr`. Оно выдает ответ в форме: координаты рамки, в которой
найден текст, сам текст и число - степень уверенности, что текст найден корректно. `CONFIDENCE_LEVEL = 0.4` - с такого уровня степени уверенности считаем, что текст найден корректно. Это число было подобрано исходя из поиска информации на изображении (например, может быть распознан текст на фоне, а не в рамке, но у него будет `CONFIDENCE_LEVEL < 0.4`)  
  
Для всплывающего окна с информацией о бое нужно кроме текста определить цвет бойца и номер раунда. Также как с поиском `min_intensity` 
и `max_intensity`, происходил поиск средней интенсивности в квадратах, которые отвечают за номер раунда. И определен порог уровня, 
при котором цвет бойца - красный, а когда синий.  
  
Все всплывающие окна, кроме информации о бое - окна, появляющиеся на какое-то количество секунд (точно больше 4 секунд), и информация 
в них не обновяется. Поэтому в этих окнах на 3 секунде после того как они появились (потому что на первой секунде текст может 
только появляться), считывается информация с них один раз, а заносится информация только когда окно пропадает, чтобы можно
было вписать величину "duration" - количество секунд, которое высвечивалось всплывающее окно. 
  
  
*Возможно корректировать и масштабировать найденные окна на другие разрешения, но пока рассматривалась упрощенная задача
с одним разрешением видео  

In [ ]:
INFO_COORDINATES = [
    [
        (slice(606, 683), slice(173, 377)), (slice(606, 643), slice(380, 654)), (slice(646, 683), slice(380, 654))
    ],
    [
        (slice(209, 250), slice(503, 590)), (slice(209, 250), slice(685, 767)),
        (slice(279, 329), slice(503, 590)), (slice(279, 329), slice(685, 767)),
        (slice(350, 400), slice(503, 590)), (slice(350, 400), slice(685, 767)),
        (slice(421, 470), slice(503, 590)), (slice(421, 470), slice(685, 767)),
        (slice(470, 587), slice(3, 496)), (slice(470, 587), slice(775, 1276)),
        (slice(590, 616), slice(10, 496)), (slice(590, 616), slice(775, 1276))
    ],
    [
        [(slice(641, 682), slice(562, 709))],
        [(slice(656, 667), slice(530, 541)), (slice(656, 667), slice(724, 735))],
        [(slice(628, 633), slice(617, 622)), (slice(628, 633), slice(630, 635)), (slice(628, 633), slice(643, 648))],
        [(slice(641, 882), slice(254, 525)), (slice(641, 882), slice(739, 1013))]
    ],
    [
        (slice(606, 655), slice(149, 435))
    ]
]

In [ ]:
# patterns for finding white lines
PATTERNS_COORDINATES = (((slice(606, 683), slice(169, 172)), (slice(606, 683), slice(169 + 3, 172 + 3))),
                        ((slice(208, 212), slice(503, 590)), (slice(208 - 4, 212 - 4), slice(503, 590))),
                        ((slice(640, 680), slice(557, 560)), (slice(640, 680), slice(557 + 3, 560 + 3))),
                        ((slice(606, 656), slice(145, 148)), (slice(606, 656), slice(145 - 3, 148 - 3))))

# patterns for min/max of white lines
PATTERNS_MIN_MAX = np.array([[120, 250],
                             [135, 160],
                             [115, 150],
                             [115, 200]])

# Анализ результатов работы main.py скрипта (file_res.json) для данного видео

Видео: https://www.youtube.com/watch?v=byl-_DM5kgs&ab_channel=SpartacusSports

Сгенерирован json файл, в котором есть соответствующая структура: ключ - время на видео, значение - информация о всплывающем окне, 
которое появляется в данное время на видео.  
На видео были представлены все возможные типы всплывающих окон. 
Длительность видео (M:S)=22:56, а время извлечения информации 378 секунд с использованием CPU.

file_res.json:  
```json
{
  "00:12": {
    "type": "primary_fighter_info",
    "name": "KAYQUE DE PAULA",
    "record": "2-0-0",
    "place": "BARRA DO PIRAI BRAZIL",
    "duration": 20
  },
  "00:39": {
    "type": "primary_fighter_info",
    "name": "WANDERSON SANTOS",
    "record": "1-3-0",
    "place": "FLORIANO BRAZIL",
    "duration": 20
  },
  "01:07": {
    "type": "extensive_fighter_info",
    "fighter_1": {
      "color": "red",
      "name": "WANDERSON SANTOS",
      "country": "BRAZIL",
      "record": "1-3-0",
      "age": "22",
      "height": "175",
      "weight": "61"
    },
    "fighter_2": {
      "color": "blue",
      "name": "KAYQUE DE PAULA",
      "country": "BRAZIL",
      "record": "2-0-0",
      "age": "19",
      "height": "174",
      "weight": "61"
    },
    "duration": 9
  },
  "01:22": {
    "type": "fight_info",
    "time": "4:59",
    "round": 1
  },
  "01:23": {
    "type": "fight_info",
    "time": "4:58",
    "round": 1
  },
  "01:24": {
    "type": "fight_info",
    "time": "4:57",
    "round": 1
  },
  "01:25": {
    "type": "fight_info",
    "time": "4:56",
    "round": 1
  },
  "01:26": {
    "type": "fight_info",
    "time": "4:55",
    "round": 1
  },
  "01:27": {
    "type": "fight_info",
    "time": "4:54",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "red"
    }
  },
  "01:28": {
    "type": "fight_info",
    "time": "4:53",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:29": {
    "type": "fight_info",
    "time": "4:52",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:30": {
    "type": "fight_info",
    "time": "4:51",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:31": {
    "type": "fight_info",
    "time": "4:50",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:32": {
    "type": "fight_info",
    "time": "4:49",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:33": {
    "type": "fight_info",
    "time": "4:48",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:34": {
    "type": "fight_info",
    "time": "4:47",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:35": {
    "type": "fight_info",
    "time": "4:46",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:36": {
    "type": "fight_info",
    "time": "4:45",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:37": {
    "type": "fight_info",
    "time": "4:44",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:38": {
    "type": "fight_info",
    "time": "4:43",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:39": {
    "type": "fight_info",
    "time": "4:42",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:40": {
    "type": "fight_info",
    "time": "4:41",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:41": {
    "type": "fight_info",
    "time": "4:40",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:42": {
    "type": "fight_info",
    "time": "4:39",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:43": {
    "type": "fight_info",
    "time": "4:38",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:44": {
    "type": "fight_info",
    "time": "4:37",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:45": {
    "type": "fight_info",
    "time": "4:36",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:46": {
    "type": "fight_info",
    "time": "4:35",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:47": {
    "type": "fight_info",
    "time": "4:34",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:48": {
    "type": "fight_info",
    "time": "4:33",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:49": {
    "type": "fight_info",
    "time": "4:32",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:50": {
    "type": "fight_info",
    "time": "4:31",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:51": {
    "type": "fight_info",
    "time": "4:30",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:52": {
    "type": "fight_info",
    "time": "4:29",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:53": {
    "type": "fight_info",
    "time": "4:28",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:54": {
    "type": "fight_info",
    "time": "4:27",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:55": {
    "type": "fight_info",
    "time": "4:26",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:56": {
    "type": "fight_info",
    "time": "4:25",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:57": {
    "type": "fight_info",
    "time": "4:24",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:58": {
    "type": "fight_info",
    "time": "4:23",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "01:59": {
    "type": "fight_info",
    "time": "4:22",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:00": {
    "type": "fight_info",
    "time": "4:21",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:01": {
    "type": "fight_info",
    "time": "4:20",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:02": {
    "type": "fight_info",
    "time": "4:19",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:03": {
    "type": "fight_info",
    "time": "4:18",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:04": {
    "type": "fight_info",
    "time": "4:17",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:05": {
    "type": "fight_info",
    "time": "4:16",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:06": {
    "type": "fight_info",
    "time": "4:15",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:07": {
    "type": "fight_info",
    "time": "4:14",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:08": {
    "type": "fight_info",
    "time": "4:13",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:09": {
    "type": "fight_info",
    "time": "4:12",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:10": {
    "type": "fight_info",
    "time": "4:11",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:11": {
    "type": "fight_info",
    "time": "4:10",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:12": {
    "type": "fight_info",
    "time": "4:09",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:13": {
    "type": "fight_info",
    "time": "4:08",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:14": {
    "type": "fight_info",
    "time": "4:07",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:15": {
    "type": "fight_info",
    "time": "4:06",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:16": {
    "type": "fight_info",
    "time": "4:05",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:17": {
    "type": "fight_info",
    "time": "4:04",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:18": {
    "type": "fight_info",
    "time": "4:03",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:19": {
    "type": "fight_info",
    "time": "4:02",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:20": {
    "type": "fight_info",
    "time": "4:01",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:21": {
    "type": "fight_info",
    "time": "4:00",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:22": {
    "type": "fight_info",
    "time": "3:59",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:23": {
    "type": "fight_info",
    "time": "3:58",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:24": {
    "type": "fight_info",
    "time": "3:57",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:25": {
    "type": "fight_info",
    "time": "3:56",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:26": {
    "type": "fight_info",
    "time": "3:55",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:27": {
    "type": "fight_info",
    "time": "3:54",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:28": {
    "type": "fight_info",
    "time": "3:53",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:29": {
    "type": "fight_info",
    "time": "3:52",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:30": {
    "type": "fight_info",
    "time": "3:51",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:31": {
    "type": "fight_info",
    "time": "3:50",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:32": {
    "type": "fight_info",
    "time": "3:49",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:33": {
    "type": "fight_info",
    "time": "3:48",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:34": {
    "type": "fight_info",
    "time": "3:47",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:35": {
    "type": "fight_info",
    "time": "3:46",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:36": {
    "type": "fight_info",
    "time": "3:45",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:37": {
    "type": "fight_info",
    "time": "3:44",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:38": {
    "type": "fight_info",
    "time": "3:43",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:39": {
    "type": "fight_info",
    "time": "3:42",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:40": {
    "type": "fight_info",
    "time": "3:41",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:41": {
    "type": "fight_info",
    "time": "3:40",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:42": {
    "type": "fight_info",
    "time": "3:39",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:43": {
    "type": "fight_info",
    "time": "3:38",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:44": {
    "type": "fight_info",
    "time": "3:37",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:45": {
    "type": "fight_info",
    "time": "3:36",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:46": {
    "type": "fight_info",
    "time": "3:35",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:47": {
    "type": "fight_info",
    "time": "3:34",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:48": {
    "type": "fight_info",
    "time": "3:33",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:49": {
    "type": "fight_info",
    "time": "3:32",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:50": {
    "type": "fight_info",
    "time": "3:31",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:51": {
    "type": "fight_info",
    "time": "3:30",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:52": {
    "type": "fight_info",
    "time": "3:29",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:53": {
    "type": "fight_info",
    "time": "3:28",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:54": {
    "type": "fight_info",
    "time": "3:27",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:55": {
    "type": "fight_info",
    "time": "3:26",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:56": {
    "type": "fight_info",
    "time": "3:25",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:57": {
    "type": "fight_info",
    "time": "3:24",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:58": {
    "type": "fight_info",
    "time": "3:23",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "02:59": {
    "type": "fight_info",
    "time": "3:22",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:00": {
    "type": "fight_info",
    "time": "3:21",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:01": {
    "type": "fight_info",
    "time": "3:20",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:02": {
    "type": "fight_info",
    "time": "3:19",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:03": {
    "type": "fight_info",
    "time": "3:18",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:04": {
    "type": "fight_info",
    "time": "3:17",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:05": {
    "type": "fight_info",
    "time": "3:16",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:06": {
    "type": "fight_info",
    "time": "3:15",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:07": {
    "type": "fight_info",
    "time": "3:14",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:08": {
    "type": "fight_info",
    "time": "3:13",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:09": {
    "type": "fight_info",
    "time": "3:12",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:10": {
    "type": "fight_info",
    "time": "3:11",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:11": {
    "type": "fight_info",
    "time": "3:10",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:12": {
    "type": "fight_info",
    "time": "3:09",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:13": {
    "type": "fight_info",
    "time": "3:08",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:14": {
    "type": "fight_info",
    "time": "3:07",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:15": {
    "type": "fight_info",
    "time": "3:06",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:16": {
    "type": "fight_info",
    "time": "3:05",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:17": {
    "type": "fight_info",
    "time": "3:04",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:18": {
    "type": "fight_info",
    "time": "3:03",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:19": {
    "type": "fight_info",
    "time": "3:02",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:20": {
    "type": "fight_info",
    "time": "3:01",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:21": {
    "type": "fight_info",
    "time": "3:00",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:22": {
    "type": "fight_info",
    "time": "2:59",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:23": {
    "type": "fight_info",
    "time": "2:58",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:24": {
    "type": "fight_info",
    "time": "2:57",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:25": {
    "type": "fight_info",
    "time": "2:56",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:26": {
    "type": "fight_info",
    "time": "2:55",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:27": {
    "type": "fight_info",
    "time": "2:54",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:28": {
    "type": "fight_info",
    "time": "2:53",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:29": {
    "type": "fight_info",
    "time": "2:52",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:30": {
    "type": "fight_info",
    "time": "2:51",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:31": {
    "type": "fight_info",
    "time": "2:50",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:32": {
    "type": "fight_info",
    "time": "2:49",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:33": {
    "type": "fight_info",
    "time": "2:48",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:34": {
    "type": "fight_info",
    "time": "2:47",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:35": {
    "type": "fight_info",
    "time": "2:46",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:36": {
    "type": "fight_info",
    "time": "2:45",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:37": {
    "type": "fight_info",
    "time": "2:44",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:38": {
    "type": "fight_info",
    "time": "2:43",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:39": {
    "type": "fight_info",
    "time": "2:42",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:40": {
    "type": "fight_info",
    "time": "2:41",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:41": {
    "type": "fight_info",
    "time": "2:40",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:42": {
    "type": "fight_info",
    "time": "2:39",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:43": {
    "type": "fight_info",
    "time": "2:38",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:44": {
    "type": "fight_info",
    "time": "2:37",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:45": {
    "type": "fight_info",
    "time": "2:36",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:46": {
    "type": "fight_info",
    "time": "2:35",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:47": {
    "type": "fight_info",
    "time": "2:34",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:48": {
    "type": "fight_info",
    "time": "2:33",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:49": {
    "type": "fight_info",
    "time": "2:32",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:50": {
    "type": "fight_info",
    "time": "2:31",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:51": {
    "type": "fight_info",
    "time": "2:30",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:52": {
    "type": "fight_info",
    "time": "2:29",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:53": {
    "type": "fight_info",
    "time": "2:28",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:54": {
    "type": "fight_info",
    "time": "2:27",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:55": {
    "type": "fight_info",
    "time": "2:26",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:56": {
    "type": "fight_info",
    "time": "2:25",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:57": {
    "type": "fight_info",
    "time": "2:24",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:58": {
    "type": "fight_info",
    "time": "2:23",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "03:59": {
    "type": "fight_info",
    "time": "2:22",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:00": {
    "type": "fight_info",
    "time": "2:21",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:01": {
    "type": "fight_info",
    "time": "2:20",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:02": {
    "type": "fight_info",
    "time": "2:19",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:03": {
    "type": "fight_info",
    "time": "2:18",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:04": {
    "type": "fight_info",
    "time": "2:17",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:05": {
    "type": "fight_info",
    "time": "2:16",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:06": {
    "type": "fight_info",
    "time": "2:15",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:07": {
    "type": "fight_info",
    "time": "2:14",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:08": {
    "type": "fight_info",
    "time": "2:13",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:09": {
    "type": "fight_info",
    "time": "2:12",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:10": {
    "type": "fight_info",
    "time": "2:11",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:11": {
    "type": "fight_info",
    "time": "2:10",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:12": {
    "type": "fight_info",
    "time": "2:09",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:13": {
    "type": "fight_info",
    "time": "2:08",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:14": {
    "type": "fight_info",
    "time": "2:07",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:15": {
    "type": "fight_info",
    "time": "2:06",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:16": {
    "type": "fight_info",
    "time": "2:05",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:17": {
    "type": "fight_info",
    "time": "2:04",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:18": {
    "type": "fight_info",
    "time": "2:03",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:19": {
    "type": "fight_info",
    "time": "2:02",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:20": {
    "type": "fight_info",
    "time": "2:01",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:21": {
    "type": "fight_info",
    "time": "2:00",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:22": {
    "type": "fight_info",
    "time": "1:59",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:23": {
    "type": "fight_info",
    "time": "1:58",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:24": {
    "type": "fight_info",
    "time": "1:57",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:25": {
    "type": "fight_info",
    "time": "1:56",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:26": {
    "type": "fight_info",
    "time": "1:55",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:27": {
    "type": "fight_info",
    "time": "1:54",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:28": {
    "type": "fight_info",
    "time": "1:53",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:29": {
    "type": "fight_info",
    "time": "1:52",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:30": {
    "type": "fight_info",
    "time": "1:51",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:31": {
    "type": "fight_info",
    "time": "1:50",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:32": {
    "type": "fight_info",
    "time": "1:49",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:33": {
    "type": "fight_info",
    "time": "1:48",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:34": {
    "type": "fight_info",
    "time": "1:47",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:35": {
    "type": "fight_info",
    "time": "1:46",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:36": {
    "type": "fight_info",
    "time": "1:45",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:37": {
    "type": "fight_info",
    "time": "1:44",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:38": {
    "type": "fight_info",
    "time": "1:43",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:39": {
    "type": "fight_info",
    "time": "1:42",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:40": {
    "type": "fight_info",
    "time": "1:41",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:41": {
    "type": "fight_info",
    "time": "1:40",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:42": {
    "type": "fight_info",
    "time": "1:39",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:43": {
    "type": "fight_info",
    "time": "1:38",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:44": {
    "type": "fight_info",
    "time": "1837",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:45": {
    "type": "fight_info",
    "time": "1:36",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:46": {
    "type": "fight_info",
    "time": "1:35",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:47": {
    "type": "fight_info",
    "time": "1:34",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:48": {
    "type": "fight_info",
    "time": "1:33",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:49": {
    "type": "fight_info",
    "time": "1:32",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:50": {
    "type": "fight_info",
    "time": "1:31",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:51": {
    "type": "fight_info",
    "time": "1:30",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:52": {
    "type": "fight_info",
    "time": "1:29",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:53": {
    "type": "fight_info",
    "time": "1:28",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:54": {
    "type": "fight_info",
    "time": "1:27",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:55": {
    "type": "fight_info",
    "time": "1:26",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:56": {
    "type": "fight_info",
    "time": "1:25",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:57": {
    "type": "fight_info",
    "time": "1:24",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:58": {
    "type": "fight_info",
    "time": "1:23",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "04:59": {
    "type": "fight_info",
    "time": "1:22",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:00": {
    "type": "fight_info",
    "time": "1:21",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:01": {
    "type": "fight_info",
    "time": "1:20",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:02": {
    "type": "fight_info",
    "time": "1:19",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:03": {
    "type": "fight_info",
    "time": "1:18",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:04": {
    "type": "fight_info",
    "time": "1:17",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:05": {
    "type": "fight_info",
    "time": "1:16",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:06": {
    "type": "fight_info",
    "time": "1:15",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:07": {
    "type": "fight_info",
    "time": "1:14",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:08": {
    "type": "fight_info",
    "time": "1:13",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:09": {
    "type": "fight_info",
    "time": "1:12",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:10": {
    "type": "fight_info",
    "time": "1:11",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:11": {
    "type": "fight_info",
    "time": "1:10",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:12": {
    "type": "fight_info",
    "time": "1:09",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:13": {
    "type": "fight_info",
    "time": "1:08",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:14": {
    "type": "fight_info",
    "time": "1:07",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:15": {
    "type": "fight_info",
    "time": "1:06",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:16": {
    "type": "fight_info",
    "time": "1:05",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:17": {
    "type": "fight_info",
    "time": "1:04",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:18": {
    "type": "fight_info",
    "time": "1:03",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:19": {
    "type": "fight_info",
    "time": "1:02",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:20": {
    "type": "fight_info",
    "time": "1:01",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:21": {
    "type": "fight_info",
    "time": "1:00",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:22": {
    "type": "fight_info",
    "time": "0:59",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:23": {
    "type": "fight_info",
    "time": "0:58",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:24": {
    "type": "fight_info",
    "time": "0:57",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:25": {
    "type": "fight_info",
    "time": "0:56",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:26": {
    "type": "fight_info",
    "time": "0:55",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:27": {
    "type": "fight_info",
    "time": "0:54",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:28": {
    "type": "fight_info",
    "time": "0:53",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:29": {
    "type": "fight_info",
    "time": "0:52",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:30": {
    "type": "fight_info",
    "time": "0:51",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:31": {
    "type": "fight_info",
    "time": "0:50",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:32": {
    "type": "fight_info",
    "time": "0:49",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:33": {
    "type": "fight_info",
    "time": "0:48",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:34": {
    "type": "fight_info",
    "time": "0:47",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:35": {
    "type": "fight_info",
    "time": "0:46",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:36": {
    "type": "fight_info",
    "time": "0:45",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:37": {
    "type": "fight_info",
    "time": "0:44",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:38": {
    "type": "fight_info",
    "time": "0:43",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:39": {
    "type": "fight_info",
    "time": "0:42",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:40": {
    "type": "fight_info",
    "time": "0:41",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:41": {
    "type": "fight_info",
    "time": "0:40",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:42": {
    "type": "fight_info",
    "time": "0:39",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:43": {
    "type": "fight_info",
    "time": "0:38",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:44": {
    "type": "fight_info",
    "time": "0:37",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:45": {
    "type": "fight_info",
    "time": "0:36",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:46": {
    "type": "fight_info",
    "time": "0:35",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:47": {
    "type": "fight_info",
    "time": "0:34",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:48": {
    "type": "fight_info",
    "time": "0:33",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:49": {
    "type": "fight_info",
    "time": "0:32",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:50": {
    "type": "fight_info",
    "time": "0:31",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:51": {
    "type": "fight_info",
    "time": "0:30",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:52": {
    "type": "fight_info",
    "time": "0:29",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:53": {
    "type": "fight_info",
    "time": "0:28",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:54": {
    "type": "fight_info",
    "time": "0:27",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:55": {
    "type": "fight_info",
    "time": "0:26",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:56": {
    "type": "fight_info",
    "time": "0:25",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:57": {
    "type": "fight_info",
    "time": "0:24",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:58": {
    "type": "fight_info",
    "time": "0:23",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "05:59": {
    "type": "fight_info",
    "time": "0:22",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "06:00": {
    "type": "fight_info",
    "time": "0:21",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "06:01": {
    "type": "fight_info",
    "time": "0:20",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "06:02": {
    "type": "fight_info",
    "time": "0:19",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "06:03": {
    "type": "fight_info",
    "time": "0:18",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "06:04": {
    "type": "fight_info",
    "time": "0:17",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "06:05": {
    "type": "fight_info",
    "time": "0:16",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "06:06": {
    "type": "fight_info",
    "time": "0:15",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "06:07": {
    "type": "fight_info",
    "time": "0:14",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "06:08": {
    "type": "fight_info",
    "time": "0:13",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "06:09": {
    "type": "fight_info",
    "time": "0:12",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "06:10": {
    "type": "fight_info",
    "time": "0:11",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "06:11": {
    "type": "fight_info",
    "time": "0:10",
    "round": 1,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "06:12": {
    "type": "fight_info",
    "time": "0:09",
    "round": 1
  },
  "06:53": {
    "type": "advertising",
    "text": "SUBSCRIBE IG: @SPARTACUSAPP",
    "duration": 15
  },
  "07:43": {
    "type": "fight_info",
    "time": "5:00",
    "round": 2
  },
  "07:44": {
    "type": "fight_info",
    "time": "4:59",
    "round": 2
  },
  "07:45": {
    "type": "fight_info",
    "time": "4:58",
    "round": 2
  },
  "07:46": {
    "type": "fight_info",
    "time": "4:57",
    "round": 2
  },
  "07:47": {
    "type": "fight_info",
    "time": "4:56",
    "round": 2
  },
  "07:48": {
    "type": "fight_info",
    "time": "4:55",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE ",
      "color": "red"
    }
  },
  "07:49": {
    "type": "fight_info",
    "time": "4:54",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "07:50": {
    "type": "fight_info",
    "time": "4:53",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "07:51": {
    "type": "fight_info",
    "time": "4:52",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "07:52": {
    "type": "fight_info",
    "time": "4:51",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "07:53": {
    "type": "fight_info",
    "time": "4:50",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "07:54": {
    "type": "fight_info",
    "time": "4:49",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "07:55": {
    "type": "fight_info",
    "time": "4:48",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "07:56": {
    "type": "fight_info",
    "time": "4:47",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "07:57": {
    "type": "fight_info",
    "time": "4:46",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "07:58": {
    "type": "fight_info",
    "time": "4:45",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:00": {
    "type": "fight_info",
    "time": "4:43",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:01": {
    "type": "fight_info",
    "time": "4:42",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:02": {
    "type": "fight_info",
    "time": "4:41",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:03": {
    "type": "fight_info",
    "time": "4:40",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:04": {
    "type": "fight_info",
    "time": "4:39",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:05": {
    "type": "fight_info",
    "time": "4:38",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:06": {
    "type": "fight_info",
    "time": "4:37",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:07": {
    "type": "fight_info",
    "time": "4:36",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:08": {
    "type": "fight_info",
    "time": "4:35",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:09": {
    "type": "fight_info",
    "time": "4:34",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:10": {
    "type": "fight_info",
    "time": "4:33",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:11": {
    "type": "fight_info",
    "time": "4:32",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:12": {
    "type": "fight_info",
    "time": "4:31",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:13": {
    "type": "fight_info",
    "time": "4:30",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:14": {
    "type": "fight_info",
    "time": "4:29",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:15": {
    "type": "fight_info",
    "time": "4:28",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:16": {
    "type": "fight_info",
    "time": "4:27",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:17": {
    "type": "fight_info",
    "time": "4:26",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:18": {
    "type": "fight_info",
    "time": "4:25",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:19": {
    "type": "fight_info",
    "time": "4:24",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:20": {
    "type": "fight_info",
    "time": "4:23",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:21": {
    "type": "fight_info",
    "time": "4:22",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:22": {
    "type": "fight_info",
    "time": "4:21",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:23": {
    "type": "fight_info",
    "time": "4:20",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:24": {
    "type": "fight_info",
    "time": "4:19",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:25": {
    "type": "fight_info",
    "time": "4:18",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:26": {
    "type": "fight_info",
    "time": "4:17",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:27": {
    "type": "fight_info",
    "time": "4:16",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:28": {
    "type": "fight_info",
    "time": "4:15",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:29": {
    "type": "fight_info",
    "time": "4:14",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:30": {
    "type": "fight_info",
    "time": "4:13",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:31": {
    "type": "fight_info",
    "time": "4:12",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:32": {
    "type": "fight_info",
    "time": "4:11",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:33": {
    "type": "fight_info",
    "time": "4:10",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:34": {
    "type": "fight_info",
    "time": "4:09",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:35": {
    "type": "fight_info",
    "time": "4:08",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:36": {
    "type": "fight_info",
    "time": "4:07",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:37": {
    "type": "fight_info",
    "time": "4:06",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:38": {
    "type": "fight_info",
    "time": "4:05",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:39": {
    "type": "fight_info",
    "time": "4:04",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:40": {
    "type": "fight_info",
    "time": "4:03",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:41": {
    "type": "fight_info",
    "time": "4:02",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:42": {
    "type": "fight_info",
    "time": "4:01",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:43": {
    "type": "fight_info",
    "time": "4:00",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:44": {
    "type": "fight_info",
    "time": "3:59",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:45": {
    "type": "fight_info",
    "time": "3:58",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:46": {
    "type": "fight_info",
    "time": "3:57",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:47": {
    "type": "fight_info",
    "time": "3:56",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:48": {
    "type": "fight_info",
    "time": "3:55",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:49": {
    "type": "fight_info",
    "time": "3:54",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:50": {
    "type": "fight_info",
    "time": "3:53",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:51": {
    "type": "fight_info",
    "time": "3:52",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:52": {
    "type": "fight_info",
    "time": "3:51",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:53": {
    "type": "fight_info",
    "time": "3:50",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:54": {
    "type": "fight_info",
    "time": "3:49",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:55": {
    "type": "fight_info",
    "time": "3:48",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:56": {
    "type": "fight_info",
    "time": "3:47",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:57": {
    "type": "fight_info",
    "time": "3:46",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:58": {
    "type": "fight_info",
    "time": "345",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "08:59": {
    "type": "fight_info",
    "time": "3:44",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:00": {
    "type": "fight_info",
    "time": "3:43",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:01": {
    "type": "fight_info",
    "time": "3:42",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:02": {
    "type": "fight_info",
    "time": "3:41",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:03": {
    "type": "fight_info",
    "time": "3:40",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:04": {
    "type": "fight_info",
    "time": "3:39",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:05": {
    "type": "fight_info",
    "time": "3:38",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:06": {
    "type": "fight_info",
    "time": "3:37",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:07": {
    "type": "fight_info",
    "time": "3:36",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:08": {
    "type": "fight_info",
    "time": "3:35",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:09": {
    "type": "fight_info",
    "time": "3:34",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:10": {
    "type": "fight_info",
    "time": "3:33",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:11": {
    "type": "fight_info",
    "time": "3:32",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:12": {
    "type": "fight_info",
    "time": "3:31",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:13": {
    "type": "fight_info",
    "time": "3:30",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:14": {
    "type": "fight_info",
    "time": "3:29",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:15": {
    "type": "fight_info",
    "time": "3:28",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:16": {
    "type": "fight_info",
    "time": "3:27",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:17": {
    "type": "fight_info",
    "time": "3:26",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:18": {
    "type": "fight_info",
    "time": "3:25",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:19": {
    "type": "fight_info",
    "time": "3:24",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:20": {
    "type": "fight_info",
    "time": "3:23",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:21": {
    "type": "fight_info",
    "time": "3:22",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:22": {
    "type": "fight_info",
    "time": "3:21",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:23": {
    "type": "fight_info",
    "time": "3:20",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:24": {
    "type": "fight_info",
    "time": "3:19",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:25": {
    "type": "fight_info",
    "time": "3:18",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:26": {
    "type": "fight_info",
    "time": "3:17",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:27": {
    "type": "fight_info",
    "time": "3:16",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:28": {
    "type": "fight_info",
    "time": "3:15",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:29": {
    "type": "fight_info",
    "time": "3:14",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:30": {
    "type": "fight_info",
    "time": "3:13",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:31": {
    "type": "fight_info",
    "time": "3:12",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:32": {
    "type": "fight_info",
    "time": "3:11",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:33": {
    "type": "fight_info",
    "time": "3:10",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:34": {
    "type": "fight_info",
    "time": "3:09",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:35": {
    "type": "fight_info",
    "time": "3:08",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:36": {
    "type": "fight_info",
    "time": "3:07",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:37": {
    "type": "fight_info",
    "time": "3:06",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:38": {
    "type": "fight_info",
    "time": "3:05",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:39": {
    "type": "fight_info",
    "time": "3:04",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:40": {
    "type": "fight_info",
    "time": "3:03",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:41": {
    "type": "fight_info",
    "time": "3:02",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:42": {
    "type": "fight_info",
    "time": "3:01",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:43": {
    "type": "fight_info",
    "time": "3:00",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:44": {
    "type": "fight_info",
    "time": "2:59",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:45": {
    "type": "fight_info",
    "time": "2:58",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:46": {
    "type": "fight_info",
    "time": "2:57",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:47": {
    "type": "fight_info",
    "time": "2:56",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:48": {
    "type": "fight_info",
    "time": "2:55",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:49": {
    "type": "fight_info",
    "time": "2:54",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:50": {
    "type": "fight_info",
    "time": "2:53",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:51": {
    "type": "fight_info",
    "time": "2:52",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:52": {
    "type": "fight_info",
    "time": "2:51",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:53": {
    "type": "fight_info",
    "time": "2:50",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:54": {
    "type": "fight_info",
    "time": "2:49",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:55": {
    "type": "fight_info",
    "time": "2:48",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:56": {
    "type": "fight_info",
    "time": "2:47",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:57": {
    "type": "fight_info",
    "time": "2:46",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:58": {
    "type": "fight_info",
    "time": "2:45",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "09:59": {
    "type": "fight_info",
    "time": "2:44",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:00": {
    "type": "fight_info",
    "time": "2:43",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:01": {
    "type": "fight_info",
    "time": "2:42",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:02": {
    "type": "fight_info",
    "time": "2:41",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:03": {
    "type": "fight_info",
    "time": "2:40",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:04": {
    "type": "fight_info",
    "time": "2:39",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:05": {
    "type": "fight_info",
    "time": "2:38",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:06": {
    "type": "fight_info",
    "time": "2:37",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:07": {
    "type": "fight_info",
    "time": "2:36",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:08": {
    "type": "fight_info",
    "time": "2:35",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:09": {
    "type": "fight_info",
    "time": "2:34",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:10": {
    "type": "fight_info",
    "time": "2:33",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:11": {
    "type": "fight_info",
    "time": "2:32",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:12": {
    "type": "fight_info",
    "time": "2:31",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:13": {
    "type": "fight_info",
    "time": "2:30",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:14": {
    "type": "fight_info",
    "time": "2:29",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:15": {
    "type": "fight_info",
    "time": "2:28",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:16": {
    "type": "fight_info",
    "time": "2:27",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:17": {
    "type": "fight_info",
    "time": "2:26",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:18": {
    "type": "fight_info",
    "time": "2:25",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:19": {
    "type": "fight_info",
    "time": "2:24",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:20": {
    "type": "fight_info",
    "time": "2:23",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:21": {
    "type": "fight_info",
    "time": "2:22",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:22": {
    "type": "fight_info",
    "time": "2:21",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:23": {
    "type": "fight_info",
    "time": "2:20",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:24": {
    "type": "fight_info",
    "time": "2:19",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:25": {
    "type": "fight_info",
    "time": "2:18",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:26": {
    "type": "fight_info",
    "time": "2:17",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:27": {
    "type": "fight_info",
    "time": "2:16",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:28": {
    "type": "fight_info",
    "time": "2:15",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:29": {
    "type": "fight_info",
    "time": "2:14",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:30": {
    "type": "fight_info",
    "time": "2:13",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:31": {
    "type": "fight_info",
    "time": "2:12",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:32": {
    "type": "fight_info",
    "time": "2:11",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:33": {
    "type": "fight_info",
    "time": "2:10",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:34": {
    "type": "fight_info",
    "time": "2:09",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:35": {
    "type": "fight_info",
    "time": "2:08",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:36": {
    "type": "fight_info",
    "time": "2:07",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:37": {
    "type": "fight_info",
    "time": "2:06",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:38": {
    "type": "fight_info",
    "time": "2:05",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:39": {
    "type": "fight_info",
    "time": "2:04",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:40": {
    "type": "fight_info",
    "time": "2:03",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:41": {
    "type": "fight_info",
    "time": "2:02",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:42": {
    "type": "fight_info",
    "time": "2:01",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:43": {
    "type": "fight_info",
    "time": "2:00",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:44": {
    "type": "fight_info",
    "time": "1:59",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:45": {
    "type": "fight_info",
    "time": "1:58",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:46": {
    "type": "fight_info",
    "time": "1:57",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:47": {
    "type": "fight_info",
    "time": "1:56",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:48": {
    "type": "fight_info",
    "time": "1:55",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:49": {
    "type": "fight_info",
    "time": "1:54",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:50": {
    "type": "fight_info",
    "time": "1:53",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:51": {
    "type": "fight_info",
    "time": "1:52",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:52": {
    "type": "fight_info",
    "time": "1:51",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:53": {
    "type": "fight_info",
    "time": "1:50",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:54": {
    "type": "fight_info",
    "time": "1:49",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:55": {
    "type": "fight_info",
    "time": "1:48",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:56": {
    "type": "fight_info",
    "time": "1:47",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:57": {
    "type": "fight_info",
    "time": "1:46",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:58": {
    "type": "fight_info",
    "time": "1:45",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "10:59": {
    "type": "fight_info",
    "time": "1:44",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:00": {
    "type": "fight_info",
    "time": "1:43",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:01": {
    "type": "fight_info",
    "time": "1:42",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:02": {
    "type": "fight_info",
    "time": "1:41",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:03": {
    "type": "fight_info",
    "time": "1:40",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:04": {
    "type": "fight_info",
    "time": "1:39",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:05": {
    "type": "fight_info",
    "time": "1:38",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:06": {
    "type": "fight_info",
    "time": "1:37",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:07": {
    "type": "fight_info",
    "time": "1:36",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:08": {
    "type": "fight_info",
    "time": "1:35",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:09": {
    "type": "fight_info",
    "time": "1:34",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:10": {
    "type": "fight_info",
    "time": "1:33",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:11": {
    "type": "fight_info",
    "time": "1:32",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:12": {
    "type": "fight_info",
    "time": "1:31",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:13": {
    "type": "fight_info",
    "time": "1:30",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:14": {
    "type": "fight_info",
    "time": "1:29",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:15": {
    "type": "fight_info",
    "time": "1:28",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:16": {
    "type": "fight_info",
    "time": "1:27",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:17": {
    "type": "fight_info",
    "time": "1:26",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:18": {
    "type": "fight_info",
    "time": "1:25",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:19": {
    "type": "fight_info",
    "time": "1:24",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:20": {
    "type": "fight_info",
    "time": "1:23",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:21": {
    "type": "fight_info",
    "time": "1:22",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:22": {
    "type": "fight_info",
    "time": "1:21",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:23": {
    "type": "fight_info",
    "time": "1:20",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:24": {
    "type": "fight_info",
    "time": "1:19",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:25": {
    "type": "fight_info",
    "time": "1:18",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:26": {
    "type": "fight_info",
    "time": "1:17",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:27": {
    "type": "fight_info",
    "time": "1:16",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:28": {
    "type": "fight_info",
    "time": "1:15",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:29": {
    "type": "fight_info",
    "time": "1:14",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS 3",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:30": {
    "type": "fight_info",
    "time": "1:13",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:31": {
    "type": "fight_info",
    "time": "1:12",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:32": {
    "type": "fight_info",
    "time": "1:11",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:33": {
    "type": "fight_info",
    "time": "1:10",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:34": {
    "type": "fight_info",
    "time": "1:09",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:35": {
    "type": "fight_info",
    "time": "1:08",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:36": {
    "type": "fight_info",
    "time": "1:07",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:37": {
    "type": "fight_info",
    "time": "1:06",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:38": {
    "type": "fight_info",
    "time": "1:05",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:39": {
    "type": "fight_info",
    "time": "1:04",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:40": {
    "type": "fight_info",
    "time": "1:03",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:41": {
    "type": "fight_info",
    "time": "1:02",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:42": {
    "type": "fight_info",
    "time": "1:01",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:43": {
    "type": "fight_info",
    "time": "1:00",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:44": {
    "type": "fight_info",
    "time": "0:59",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:45": {
    "type": "fight_info",
    "time": "0:58",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:46": {
    "type": "fight_info",
    "time": "0:57",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:47": {
    "type": "fight_info",
    "time": "0:56",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:48": {
    "type": "fight_info",
    "time": "0:55",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:49": {
    "type": "fight_info",
    "time": "0:54",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:50": {
    "type": "fight_info",
    "time": "0:53",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:51": {
    "type": "fight_info",
    "time": "0:52",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:52": {
    "type": "fight_info",
    "time": "0:51",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:53": {
    "type": "fight_info",
    "time": "0:50",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:54": {
    "type": "fight_info",
    "time": "0:49",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:55": {
    "type": "fight_info",
    "time": "0:48",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:56": {
    "type": "fight_info",
    "time": "0:47",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:57": {
    "type": "fight_info",
    "time": "0:46",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:58": {
    "type": "fight_info",
    "time": "0:45",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "11:59": {
    "type": "fight_info",
    "time": "0:44",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:00": {
    "type": "fight_info",
    "time": "0:43",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:01": {
    "type": "fight_info",
    "time": "0:42",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:02": {
    "type": "fight_info",
    "time": "0:41",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:03": {
    "type": "fight_info",
    "time": "0:40",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:04": {
    "type": "fight_info",
    "time": "0:39",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:05": {
    "type": "fight_info",
    "time": "0:38",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:06": {
    "type": "fight_info",
    "time": "0:37",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:07": {
    "type": "fight_info",
    "time": "0:36",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:08": {
    "type": "fight_info",
    "time": "0:35",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:09": {
    "type": "fight_info",
    "time": "0:34",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:10": {
    "type": "fight_info",
    "time": "0:33",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:11": {
    "type": "fight_info",
    "time": "0:32",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:12": {
    "type": "fight_info",
    "time": "0:31",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:13": {
    "type": "fight_info",
    "time": "0:30",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:14": {
    "type": "fight_info",
    "time": "0:29",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:15": {
    "type": "fight_info",
    "time": "0:28",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:16": {
    "type": "fight_info",
    "time": "0:27",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:17": {
    "type": "fight_info",
    "time": "0:26",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:18": {
    "type": "fight_info",
    "time": "0:25",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:19": {
    "type": "fight_info",
    "time": "0:24",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:20": {
    "type": "fight_info",
    "time": "0:23",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:21": {
    "type": "fight_info",
    "time": "0:22",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:22": {
    "type": "fight_info",
    "time": "0:21",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:23": {
    "type": "fight_info",
    "time": "0:20",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:24": {
    "type": "fight_info",
    "time": "0:19",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:25": {
    "type": "fight_info",
    "time": "0:18",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:26": {
    "type": "fight_info",
    "time": "0:17",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:27": {
    "type": "fight_info",
    "time": "0:16",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "12:28": {
    "type": "fight_info",
    "time": "0:15",
    "round": 2,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "red"
    }
  },
  "12:29": {
    "type": "fight_info",
    "time": "0:14",
    "round": 2
  },
  "12:30": {
    "type": "fight_info",
    "time": "0:13",
    "round": 2
  },
  "12:31": {
    "type": "fight_info",
    "time": "0:12",
    "round": 2
  },
  "12:32": {
    "type": "fight_info",
    "time": "0:11",
    "round": 2
  },
  "12:33": {
    "type": "fight_info",
    "time": "0310",
    "round": 2
  },
  "12:49": {
    "type": "advertising",
    "text": "SUBSCRIBE IG: @SPARTACUSAPP",
    "duration": 32
  },
  "14:01": {
    "type": "fight_info",
    "time": "5:00",
    "round": 3
  },
  "14:02": {
    "type": "fight_info",
    "time": "4:59",
    "round": 3
  },
  "14:03": {
    "type": "fight_info",
    "time": "4:58",
    "round": 3,
    "fighter_1": {
      "name": "32AL",
      "color": "red"
    },
    "fighter_2": {
      "name": "AC",
      "color": "red"
    }
  },
  "14:04": {
    "type": "fight_info",
    "time": "4:57",
    "round": 3
  },
  "14:05": {
    "type": "fight_info",
    "time": "4:56",
    "round": 3
  },
  "14:06": {
    "type": "fight_info",
    "time": "4:55",
    "round": 3
  },
  "14:07": {
    "type": "fight_info",
    "time": "4:54",
    "round": 3
  },
  "14:08": {
    "type": "fight_info",
    "time": "4:53",
    "round": 3
  },
  "14:09": {
    "type": "fight_info",
    "time": "4:52",
    "round": 3
  },
  "14:10": {
    "type": "fight_info",
    "time": "4:51",
    "round": 3
  },
  "14:11": {
    "type": "fight_info",
    "time": "4:50",
    "round": 3
  },
  "14:12": {
    "type": "fight_info",
    "time": "4:49",
    "round": 3
  },
  "14:13": {
    "type": "fight_info",
    "time": "4:48",
    "round": 3
  },
  "14:14": {
    "type": "fight_info",
    "time": "4:47",
    "round": 3
  },
  "14:15": {
    "type": "fight_info",
    "time": "4:46",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:16": {
    "type": "fight_info",
    "time": "4:45",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:17": {
    "type": "fight_info",
    "time": "4:44",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:18": {
    "type": "fight_info",
    "time": "4:43",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:19": {
    "type": "fight_info",
    "time": "4:42",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:20": {
    "type": "fight_info",
    "time": "4:41",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:21": {
    "type": "fight_info",
    "time": "4:40",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:22": {
    "type": "fight_info",
    "time": "4:39",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:23": {
    "type": "fight_info",
    "time": "4:38",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:24": {
    "type": "fight_info",
    "time": "4:37",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:25": {
    "type": "fight_info",
    "time": "4:36",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:26": {
    "type": "fight_info",
    "time": "4:35",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:27": {
    "type": "fight_info",
    "time": "4:34",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:28": {
    "type": "fight_info",
    "time": "4:33",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:29": {
    "type": "fight_info",
    "time": "4:32",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:30": {
    "type": "fight_info",
    "time": "4:31",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:31": {
    "type": "fight_info",
    "time": "4:30",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:32": {
    "type": "fight_info",
    "time": "4:29",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:33": {
    "type": "fight_info",
    "time": "4:28",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:34": {
    "type": "fight_info",
    "time": "4:27",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:35": {
    "type": "fight_info",
    "time": "4:26",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:36": {
    "type": "fight_info",
    "time": "4:25",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:37": {
    "type": "fight_info",
    "time": "4:24",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:38": {
    "type": "fight_info",
    "time": "4:23",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:39": {
    "type": "fight_info",
    "time": "4:22",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:40": {
    "type": "fight_info",
    "time": "4:21",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:41": {
    "type": "fight_info",
    "time": "4:20",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:42": {
    "type": "fight_info",
    "time": "4:19",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:43": {
    "type": "fight_info",
    "time": "4:18",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:44": {
    "type": "fight_info",
    "time": "4:17",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:45": {
    "type": "fight_info",
    "time": "4:16",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:46": {
    "type": "fight_info",
    "time": "4:15",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:47": {
    "type": "fight_info",
    "time": "4:14",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:48": {
    "type": "fight_info",
    "time": "4:13",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:49": {
    "type": "fight_info",
    "time": "4:12",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:50": {
    "type": "fight_info",
    "time": "4:11",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:51": {
    "type": "fight_info",
    "time": "4:10",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:52": {
    "type": "fight_info",
    "time": "4:09",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:53": {
    "type": "fight_info",
    "time": "4:08",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:54": {
    "type": "fight_info",
    "time": "4:07",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:55": {
    "type": "fight_info",
    "time": "4:06",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:56": {
    "type": "fight_info",
    "time": "4:05",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:57": {
    "type": "fight_info",
    "time": "4:04",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:58": {
    "type": "fight_info",
    "time": "4:03",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "14:59": {
    "type": "fight_info",
    "time": "4:02",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:00": {
    "type": "fight_info",
    "time": "4:01",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:01": {
    "type": "fight_info",
    "time": "4:00",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:02": {
    "type": "fight_info",
    "time": "3:59",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:03": {
    "type": "fight_info",
    "time": "3:58",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:04": {
    "type": "fight_info",
    "time": "3:57",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:05": {
    "type": "fight_info",
    "time": "3:56",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:06": {
    "type": "fight_info",
    "time": "3:55",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:07": {
    "type": "fight_info",
    "time": "3:54",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:08": {
    "type": "fight_info",
    "time": "3:53",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:09": {
    "type": "fight_info",
    "time": "3:52",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:10": {
    "type": "fight_info",
    "time": "3:51",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:11": {
    "type": "fight_info",
    "time": "3:50",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:12": {
    "type": "fight_info",
    "time": "3:49",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:13": {
    "type": "fight_info",
    "time": "3:48",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:14": {
    "type": "fight_info",
    "time": "347",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:15": {
    "type": "fight_info",
    "time": "3:46",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:16": {
    "type": "fight_info",
    "time": "345",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:17": {
    "type": "fight_info",
    "time": "3:44",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:18": {
    "type": "fight_info",
    "time": "3:43",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:19": {
    "type": "fight_info",
    "time": "3:42",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:20": {
    "type": "fight_info",
    "time": "3:41",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:21": {
    "type": "fight_info",
    "time": "3:40",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:22": {
    "type": "fight_info",
    "time": "3:39",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:23": {
    "type": "fight_info",
    "time": "3:38",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:24": {
    "type": "fight_info",
    "time": "3:37",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:25": {
    "type": "fight_info",
    "time": "3:36",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:26": {
    "type": "fight_info",
    "time": "3:35",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:27": {
    "type": "fight_info",
    "time": "3:34",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:28": {
    "type": "fight_info",
    "time": "3:33",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:29": {
    "type": "fight_info",
    "time": "3:32",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:30": {
    "type": "fight_info",
    "time": "3:31",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:31": {
    "type": "fight_info",
    "time": "3:30",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:32": {
    "type": "fight_info",
    "time": "3:29",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:33": {
    "type": "fight_info",
    "time": "3:28",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:34": {
    "type": "fight_info",
    "time": "3:27",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:35": {
    "type": "fight_info",
    "time": "3:26",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:36": {
    "type": "fight_info",
    "time": "3:25",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:37": {
    "type": "fight_info",
    "time": "3:24",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:38": {
    "type": "fight_info",
    "time": "3:23",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:39": {
    "type": "fight_info",
    "time": "3:22",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:40": {
    "type": "fight_info",
    "time": "3:21",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:41": {
    "type": "fight_info",
    "time": "3:20",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:42": {
    "type": "fight_info",
    "time": "3:19",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:43": {
    "type": "fight_info",
    "time": "3:18",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:44": {
    "type": "fight_info",
    "time": "3:17",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:45": {
    "type": "fight_info",
    "time": "3:16",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:46": {
    "type": "fight_info",
    "time": "3:15",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:47": {
    "type": "fight_info",
    "time": "3:14",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:48": {
    "type": "fight_info",
    "time": "3:13",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:49": {
    "type": "fight_info",
    "time": "3:12",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:50": {
    "type": "fight_info",
    "time": "3:11",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:51": {
    "type": "fight_info",
    "time": "3:10",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:52": {
    "type": "fight_info",
    "time": "3:09",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:53": {
    "type": "fight_info",
    "time": "3:08",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:54": {
    "type": "fight_info",
    "time": "3:07",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:55": {
    "type": "fight_info",
    "time": "3:06",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:56": {
    "type": "fight_info",
    "time": "3:05",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:57": {
    "type": "fight_info",
    "time": "3:04",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:58": {
    "type": "fight_info",
    "time": "3:03",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "15:59": {
    "type": "fight_info",
    "time": "3:02",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:00": {
    "type": "fight_info",
    "time": "3:01",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:01": {
    "type": "fight_info",
    "time": "3:00",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:02": {
    "type": "fight_info",
    "time": "2:59",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:03": {
    "type": "fight_info",
    "time": "2:58",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:04": {
    "type": "fight_info",
    "time": "2:57",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:05": {
    "type": "fight_info",
    "time": "2:56",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:06": {
    "type": "fight_info",
    "time": "2:55",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:07": {
    "type": "fight_info",
    "time": "2:54",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:08": {
    "type": "fight_info",
    "time": "2:53",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:09": {
    "type": "fight_info",
    "time": "2:52",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:10": {
    "type": "fight_info",
    "time": "2:51",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:11": {
    "type": "fight_info",
    "time": "2:50",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:12": {
    "type": "fight_info",
    "time": "2:49",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:13": {
    "type": "fight_info",
    "time": "2:48",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:14": {
    "type": "fight_info",
    "time": "2:47",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:15": {
    "type": "fight_info",
    "time": "2:46",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:16": {
    "type": "fight_info",
    "time": "2:45",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:17": {
    "type": "fight_info",
    "time": "2:44",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:18": {
    "type": "fight_info",
    "time": "2:43",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:19": {
    "type": "fight_info",
    "time": "2:42",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:20": {
    "type": "fight_info",
    "time": "2:41",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:21": {
    "type": "fight_info",
    "time": "2:40",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:22": {
    "type": "fight_info",
    "time": "2:39",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:23": {
    "type": "fight_info",
    "time": "2:38",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:24": {
    "type": "fight_info",
    "time": "2:37",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:25": {
    "type": "fight_info",
    "time": "2:36",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:26": {
    "type": "fight_info",
    "time": "2:35",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:27": {
    "type": "fight_info",
    "time": "2:34",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:28": {
    "type": "fight_info",
    "time": "2:33",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:29": {
    "type": "fight_info",
    "time": "2:32",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:30": {
    "type": "fight_info",
    "time": "2:31",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:31": {
    "type": "fight_info",
    "time": "2:30",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:32": {
    "type": "fight_info",
    "time": "2:29",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:33": {
    "type": "fight_info",
    "time": "2:28",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:34": {
    "type": "fight_info",
    "time": "2:27",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:35": {
    "type": "fight_info",
    "time": "2:26",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:36": {
    "type": "fight_info",
    "time": "2:25",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:37": {
    "type": "fight_info",
    "time": "2:24",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:38": {
    "type": "fight_info",
    "time": "2:23",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:39": {
    "type": "fight_info",
    "time": "2:22",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:40": {
    "type": "fight_info",
    "time": "2:21",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:41": {
    "type": "fight_info",
    "time": "2:20",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:42": {
    "type": "fight_info",
    "time": "2:19",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:43": {
    "type": "fight_info",
    "time": "2:18",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:44": {
    "type": "fight_info",
    "time": "2:17",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:45": {
    "type": "fight_info",
    "time": "2:16",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:46": {
    "type": "fight_info",
    "time": "2:15",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:47": {
    "type": "fight_info",
    "time": "2:14",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:48": {
    "type": "fight_info",
    "time": "2:13",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:49": {
    "type": "fight_info",
    "time": "2:12",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:50": {
    "type": "fight_info",
    "time": "2:11",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:51": {
    "type": "fight_info",
    "time": "2:10",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:52": {
    "type": "fight_info",
    "time": "2:09",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:53": {
    "type": "fight_info",
    "time": "2:08",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:54": {
    "type": "fight_info",
    "time": "2:07",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:55": {
    "type": "fight_info",
    "time": "2:06",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:56": {
    "type": "fight_info",
    "time": "2:05",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:57": {
    "type": "fight_info",
    "time": "2:04",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:58": {
    "type": "fight_info",
    "time": "2:03",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "16:59": {
    "type": "fight_info",
    "time": "2:02",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:00": {
    "type": "fight_info",
    "time": "2:01",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:01": {
    "type": "fight_info",
    "time": "2:00",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:02": {
    "type": "fight_info",
    "time": "1:59",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:03": {
    "type": "fight_info",
    "time": "1:58",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:04": {
    "type": "fight_info",
    "time": "1:57",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:05": {
    "type": "fight_info",
    "time": "1:56",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:06": {
    "type": "fight_info",
    "time": "1:55",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:07": {
    "type": "fight_info",
    "time": "1:54",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:08": {
    "type": "fight_info",
    "time": "1:53",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:09": {
    "type": "fight_info",
    "time": "1:52",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:10": {
    "type": "fight_info",
    "time": "1:51",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:11": {
    "type": "fight_info",
    "time": "1:50",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:12": {
    "type": "fight_info",
    "time": "1:49",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:13": {
    "type": "fight_info",
    "time": "1:48",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:14": {
    "type": "fight_info",
    "time": "1:47",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:15": {
    "type": "fight_info",
    "time": "1:46",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:16": {
    "type": "fight_info",
    "time": "1:45",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:17": {
    "type": "fight_info",
    "time": "1:44",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:18": {
    "type": "fight_info",
    "time": "1:43",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:19": {
    "type": "fight_info",
    "time": "1:42",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:20": {
    "type": "fight_info",
    "time": "1:41",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:21": {
    "type": "fight_info",
    "time": "1:40",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:22": {
    "type": "fight_info",
    "time": "1:39",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:23": {
    "type": "fight_info",
    "time": "1:38",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:24": {
    "type": "fight_info",
    "time": "1:37",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:25": {
    "type": "fight_info",
    "time": "1:36",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:26": {
    "type": "fight_info",
    "time": "1:35",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:27": {
    "type": "fight_info",
    "time": "1:34",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:28": {
    "type": "fight_info",
    "time": "1:33",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:29": {
    "type": "fight_info",
    "time": "1:32",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:30": {
    "type": "fight_info",
    "time": "1:31",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:31": {
    "type": "fight_info",
    "time": "1:30",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:32": {
    "type": "fight_info",
    "time": "1:29",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:33": {
    "type": "fight_info",
    "time": "1:28",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:34": {
    "type": "fight_info",
    "time": "1:27",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:35": {
    "type": "fight_info",
    "time": "1:26",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:36": {
    "type": "fight_info",
    "time": "1:25",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:37": {
    "type": "fight_info",
    "time": "1:24",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:38": {
    "type": "fight_info",
    "time": "1:23",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:39": {
    "type": "fight_info",
    "time": "1:22",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:40": {
    "type": "fight_info",
    "time": "1:21",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:41": {
    "type": "fight_info",
    "time": "1:20",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:42": {
    "type": "fight_info",
    "time": "1:19",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:43": {
    "type": "fight_info",
    "time": "1:18",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:44": {
    "type": "fight_info",
    "time": "1:17",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:45": {
    "type": "fight_info",
    "time": "1:16",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:46": {
    "type": "fight_info",
    "time": "1:15",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:47": {
    "type": "fight_info",
    "time": "1:14",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:48": {
    "type": "fight_info",
    "time": "1:13",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:49": {
    "type": "fight_info",
    "time": "1:12",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:50": {
    "type": "fight_info",
    "time": "1:11",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:51": {
    "type": "fight_info",
    "time": "1:10",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:52": {
    "type": "fight_info",
    "time": "1:09",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:53": {
    "type": "fight_info",
    "time": "1:08",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:54": {
    "type": "fight_info",
    "time": "1:07",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:55": {
    "type": "fight_info",
    "time": "1:06",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:56": {
    "type": "fight_info",
    "time": "1:05",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:57": {
    "type": "fight_info",
    "time": "1:04",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:58": {
    "type": "fight_info",
    "time": "1:03",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "17:59": {
    "type": "fight_info",
    "time": "1:02",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:00": {
    "type": "fight_info",
    "time": "1:01",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:01": {
    "type": "fight_info",
    "time": "1:00",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:02": {
    "type": "fight_info",
    "time": "0:59",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:03": {
    "type": "fight_info",
    "time": "0:58",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:04": {
    "type": "fight_info",
    "time": "0:57",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:05": {
    "type": "fight_info",
    "time": "0:56",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:06": {
    "type": "fight_info",
    "time": "0:55",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:07": {
    "type": "fight_info",
    "time": "0:54",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:08": {
    "type": "fight_info",
    "time": "0:53",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:09": {
    "type": "fight_info",
    "time": "0:52",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:10": {
    "type": "fight_info",
    "time": "0:51",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:11": {
    "type": "fight_info",
    "time": "0:50",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:12": {
    "type": "fight_info",
    "time": "0:49",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:13": {
    "type": "fight_info",
    "time": "0:48",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:14": {
    "type": "fight_info",
    "time": "0:47",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:15": {
    "type": "fight_info",
    "time": "0:46",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:16": {
    "type": "fight_info",
    "time": "0:45",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:17": {
    "type": "fight_info",
    "time": "0:44",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:18": {
    "type": "fight_info",
    "time": "0:43",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:19": {
    "type": "fight_info",
    "time": "0:42",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:20": {
    "type": "fight_info",
    "time": "0:41",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:21": {
    "type": "fight_info",
    "time": "0:40",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:22": {
    "type": "fight_info",
    "time": "0:39",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:23": {
    "type": "fight_info",
    "time": "038",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:24": {
    "type": "fight_info",
    "time": "0:37",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:25": {
    "type": "fight_info",
    "time": "0:36",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:26": {
    "type": "fight_info",
    "time": "0:35",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:27": {
    "type": "fight_info",
    "time": "0:34",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:28": {
    "type": "fight_info",
    "time": "0:33",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:29": {
    "type": "fight_info",
    "time": "0:32",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:30": {
    "type": "fight_info",
    "time": "0:31",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:31": {
    "type": "fight_info",
    "time": "0:30",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:32": {
    "type": "fight_info",
    "time": "0:29",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:33": {
    "type": "fight_info",
    "time": "0:28",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:34": {
    "type": "fight_info",
    "time": "0:27",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:35": {
    "type": "fight_info",
    "time": "0:26",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:36": {
    "type": "fight_info",
    "time": "0:25",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:37": {
    "type": "fight_info",
    "time": "0:24",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:38": {
    "type": "fight_info",
    "time": "0:23",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:39": {
    "type": "fight_info",
    "time": "0:22",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:40": {
    "type": "fight_info",
    "time": "0:21",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:41": {
    "type": "fight_info",
    "time": "0:20",
    "round": 3,
    "fighter_1": {
      "name": "WANDERSON SANTOS",
      "color": "red"
    },
    "fighter_2": {
      "name": "KAYQUE DE PAULA",
      "color": "blue"
    }
  },
  "18:42": {
    "type": "fight_info",
    "time": "0:19",
    "round": 3
  },
  "18:43": {
    "type": "fight_info",
    "time": "0:18",
    "round": 3
  },
  "18:44": {
    "type": "fight_info",
    "time": "0:17",
    "round": 3
  },
  "18:45": {
    "type": "fight_info",
    "time": "0:16",
    "round": 3
  },
  "18:46": {
    "type": "fight_info",
    "time": "0:15",
    "round": 3
  },
  "19:12": {
    "type": "advertising",
    "text": "SUBSCRIBE IG: @SPARTACUSAPP",
    "duration": 48
  },
  "21:51": {
    "type": "winner",
    "name": "WANDERSON SANTOS",
    "duration": 9
  }
}